In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

depression_us=pd.read_csv("../data/depression_us_vcl.csv",index_col=[0])

In [2]:
depression_us.select_dtypes(include=np.number).columns.tolist()

['TIPI1',
 'TIPI2',
 'TIPI3',
 'TIPI4',
 'TIPI5',
 'TIPI6',
 'TIPI7',
 'TIPI8',
 'TIPI9',
 'TIPI10',
 'education',
 'urban',
 'gender',
 'age',
 'religion',
 'orientation',
 'race',
 'married',
 'familysize',
 'VCL',
 'target']

In [3]:
depression_us.head()

,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,...,gender,age,religion,orientation,race,married,familysize,VCL,Condition,target
1,6,5,4,7,5,4,7,7,1,5,...,2,16,7,0,70,1,4,8,Severe,4
3,1,1,7,4,6,4,6,1,6,1,...,2,13,4,5,70,1,5,7,Moderate,2
5,2,1,6,1,7,7,7,2,6,7,...,2,20,4,1,70,1,4,7,Mild,1
8,1,4,5,7,5,7,6,7,1,4,...,1,16,12,2,70,1,4,8,Extremely Severe,0
10,5,3,6,6,3,4,4,7,5,7,...,1,15,6,1,60,1,1,5,Moderate,2


In [4]:
# from sklearn.utils import resample
# target_0 = depression_us[depression_us['target'] ==0]
# target_1 = depression_us[depression_us['target'] ==1]
# target_2 = depression_us[depression_us['target'] ==2]
# target_3 = depression_us[depression_us['target'] ==3]
# target_4 = depression_us[depression_us['target'] ==4]

# target_0 = resample(target_0, 
#                     replace=True,     # sample with replacement
#                     n_samples=1500,    # to match majority class
#                     random_state=88)
# # Upsample minority class
# target_1 = resample(target_1, 
#                     replace=True,     # sample with replacement
#                     n_samples=1500,    # to match majority class
#                     random_state=88)

# depression_us = pd.concat([target_0, target_1,target_2,target_3,target_4])

In [5]:
depression_us['target'].value_counts()

0    3117
3    1736
2    1386
4    1287
1     678
Name: target, dtype: int64

In [6]:
depression_us['target'] = np.where(
     depression_us['target'].between(-1, 3, inclusive=False), 
    0, 
      1
)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  


In [7]:
depression_us['target'].value_counts()

0    5181
1    3023
Name: target, dtype: int64

In [8]:
# label = 'target'
# groups = depression_us.groupby(label, group_keys=False)
# depression_us = pd.DataFrame(groups.apply(lambda x: 
# x.sample(groups.size().min()))).reset_index(drop=True)

In [9]:
#cols = ['TIPI1','TIPI2','TIPI3','TIPI4','TIPI5','TIPI6','TIPI7','TIPI8','TIPI9','TIPI10']
#depression_us[cols] = depression_us[cols].astype('category')

In [10]:
depression_us['personality']=(depression_us['TIPI2']+
                                depression_us['TIPI4']+
                                depression_us['TIPI6']+
                                depression_us['TIPI8']+
                                depression_us['TIPI10']-
                                depression_us['TIPI1']-
                                depression_us['TIPI3']-
                                depression_us['TIPI5']-
                                depression_us['TIPI7']-
                                depression_us['TIPI9'])

In [11]:
depression_us['AgeBand'] = pd.cut(depression_us['age'], 4)
depression_us[['AgeBand','target']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)


,AgeBand,target
0,"(12.931, 30.25]",0.350844
1,"(30.25, 47.5]",0.420175
2,"(47.5, 64.75]",0.434412
3,"(64.75, 82.0]",0.555556


In [12]:
depression_us['AgeBand']=0
depression_us.loc[(depression_us['age']>12) & (depression_us['age']<= 31),'AgeBand'] = 3
depression_us.loc[(depression_us['age']>31) & (depression_us['age']<= 48),'AgeBand'] = 2
depression_us.loc[(depression_us['age']>48) & (depression_us['age']<= 65),'AgeBand'] = 1
depression_us.loc[depression_us['age']>65,'AgeBand']=0

In [13]:
multi_cols=['education', 'urban', 'gender', 'religion', 'orientation',
       'race', 'married','AgeBand']


#Duplicating columns for multi value columns
depression_us = pd.get_dummies(data = depression_us, columns = multi_cols)



In [14]:
depression_us.drop(['Condition','TIPI1','TIPI2','TIPI3','TIPI4','TIPI5','TIPI6','TIPI7','TIPI8','TIPI9','TIPI10','age'],axis=1,inplace=True)

In [15]:
#depression_us=depression_us.drop(depression_us.columns[0], axis=1)

In [16]:
depression_targets=depression_us['target']
depression_targets=depression_targets.astype('category')
depression_us.drop('target',axis=1,inplace=True)

In [17]:
# INSERT CODE HERE
import numpy as np
#Finding the mean and standard deviation of x and normalizing train x 
u_x_vcl = np.mean(depression_us['VCL'])
sigma_x_vcl = np.std(depression_us['VCL'])
depression_us['VCL'] = (depression_us['VCL'] - u_x_vcl)/sigma_x_vcl

#Finding the mean and standard deviation of y and normalizing train y 
u_x_familysize = np.mean(depression_us['familysize'])
sigma_x_familysize = np.std(depression_us['familysize'])
depression_us['familysize'] = (depression_us['familysize'] - u_x_familysize)/sigma_x_familysize

In [18]:
from enum import unique
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor, RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(depression_us,depression_targets, test_size=0.25)

In [19]:
lr = LogisticRegression(multi_class='ovr',solver='liblinear',penalty='l1')
pred_lr = lr.fit(X_train, y_train).predict(X_test)

# Checking Model Metrics
# from sklearn.metrics import r2_score, mean_squared_error
# print("R2 Score: ", r2_score(y_test, pred_lr))
# print("RMSE: ", np.sqrt(mean_squared_error(y_test, pred_lr)))

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.70      0.87      0.77      1318
           1       0.58      0.32      0.41       733

    accuracy                           0.67      2051
   macro avg       0.64      0.59      0.59      2051
weighted avg       0.65      0.67      0.64      2051



In [21]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(splitter='best')

pred_dt = dt.fit(X_train, y_train).predict(X_test)

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.67      0.66      0.66      1318
           1       0.40      0.42      0.41       733

    accuracy                           0.57      2051
   macro avg       0.54      0.54      0.54      2051
weighted avg       0.58      0.57      0.57      2051



In [23]:
from sklearn.ensemble import AdaBoostClassifier

In [24]:
ada_clf_g11 = AdaBoostClassifier(n_estimators=5000,learning_rate=0.01)
ada_clf_g11.get_params()

{'algorithm': 'SAMME.R',
 'base_estimator': None,
 'learning_rate': 0.01,
 'n_estimators': 5000,
 'random_state': None}

In [25]:
ada_fit_g11 = ada_clf_g11.fit(X_train, y_train)

In [26]:
OSIy_pred_g11 = ada_fit_g11.predict(X_test)

In [27]:
print(classification_report(y_test, OSIy_pred_g11))

              precision    recall  f1-score   support

           0       0.69      0.89      0.78      1318
           1       0.59      0.28      0.38       733

    accuracy                           0.67      2051
   macro avg       0.64      0.59      0.58      2051
weighted avg       0.65      0.67      0.64      2051



In [28]:
import xgboost as xgb

In [29]:
xgb_clf_g11 = xgb.XGBClassifier(n_estimators=5000,
                                learning_rate = 0.2,
                                gamma=1)

In [30]:
xgb_fit_g11 = xgb_clf_g11.fit(X_train, y_train)

In [31]:
OSIy_xgb_pred_g11 = xgb_fit_g11.predict(X_test)

In [32]:
print(classification_report(y_test, OSIy_xgb_pred_g11))

              precision    recall  f1-score   support

           0       0.70      0.83      0.76      1318
           1       0.54      0.36      0.43       733

    accuracy                           0.66      2051
   macro avg       0.62      0.60      0.60      2051
weighted avg       0.64      0.66      0.64      2051



In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
rfc = RandomForestClassifier(n_jobs=-1, random_state=10, oob_score=True)

In [35]:
xgb_fit_g11 = rfc.fit(X_train, y_train)
OSIy_xgb_pred_g11 = xgb_fit_g11.predict(X_test)

In [36]:
print(classification_report(y_test, OSIy_xgb_pred_g11))

              precision    recall  f1-score   support

           0       0.70      0.82      0.76      1318
           1       0.54      0.38      0.44       733

    accuracy                           0.66      2051
   macro avg       0.62      0.60      0.60      2051
weighted avg       0.65      0.66      0.65      2051



In [37]:
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier().fit(X_train, y_train).predict(X_test)
print(classification_report(y_test, clf))

              precision    recall  f1-score   support

           0       0.69      0.88      0.78      1318
           1       0.59      0.30      0.40       733

    accuracy                           0.68      2051
   macro avg       0.64      0.59      0.59      2051
weighted avg       0.66      0.68      0.64      2051



In [46]:
from sklearn.ensemble import StackingClassifier
# import lightgbm
# from lightgbm import LGBMClassifier

level0 = list()
level0.append(('Tree', DecisionTreeClassifier()))
level0.append(('RF', RandomForestClassifier()))
level0.append(('ADA', AdaBoostClassifier()))
level0.append(('Ridge', RidgeClassifier()))
level0.append(("XGB", xgb.XGBClassifier()))

level1 = LogisticRegression()
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.89      0.78      1318
           1       0.60      0.31      0.41       733

    accuracy                           0.68      2051
   macro avg       0.65      0.60      0.59      2051
weighted avg       0.66      0.68      0.65      2051



In [ ]:
import pickle

# Save to file in the current working directory
pkl_filename = "app/TrainedModel/StackedPickle.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)